In [1]:
import sys
import os
import shutil

# from pathlib import path

import mhi.pscad

import random 
import torch 
import numpy as np
import pandas as pd 
from collections import deque
import matplotlib.pyplot as plt
from statistics import mean 
%matplotlib inline 





In [2]:
sys.path.append('/MG_env')
from MG_env.MG_env_v1 import *
# from MG_env_v1 import*
print_test() 


sys.path.append('/MG_agent')
from MG_agent.DQN_agent import *


sys.path.append('/MG_env')
from MG_env.PSCAD_env import *
PSCAD_env_read()



agent =  Agent(state_size = 25, action_size=4, seed=0)
file_read()

PSCAD_env was read successfully
MG_env was read successfully
PSCAD_env was read successfully
DQN_agent file was read successfully


In [3]:

projectName = "feeder4_BESS_CSI"
fortranExt = '.gf46'
destFolder = "Output_Dest_Folder"

currentDirectory=os.getcwd()
MG_env_path = currentDirectory +"\\"+"MG_env\\"
resultPath=os.path.join(currentDirectory +"\\"+"MG_env\\",destFolder)
sourceFolder = currentDirectory  +"\\"+"MG_env\\"+projectName +fortranExt
sourceFile = currentDirectory  +"\\"+projectName + ".pscx"

print(currentDirectory)
print(MG_env_path)
print(resultPath)
print(sourceFolder)
print(sourceFile)

C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\Output_Dest_Folder
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\feeder4_BESS_CSI.gf46
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\feeder4_BESS_CSI.pscx


In [4]:
# Organize Files
def get_result(sourceFolder, outputName):
    files = os.listdir(sourceFolder)
    
    # get files with outputs
    outputFiles = [os.path.join(sourceFolder, out) for out in files if '.out' in out and outputName in out]
    
    resultList =[]
    for fileName in outputFiles:
        res = pd.read_fwf(fileName, skiprows = 1, header = None, delim_whitespace = True, index_col =0)
        resultList.append(res)
    names = []
    with open(os.path.join(sourceFolder, outputName+'.inf'), 'r') as file:
        for name in file:
            name = name.split('Desc="')
            name = name[1].split('Group')
            name = name[0].split('"')
            names.append(name[0])
    
    results = pd.concat(resultList, axis = 1, ignore_index = True)
    results.columns = names
    
    return results


In [5]:
# Initiate the environment 
# learn how to call another py files i your main
## Define the model topology
model_topo = {
    'BUS_1' : {'Voltage' : ['V_BUS1_A', 'V_BUS1_B', 'V_BUS1_C'] ,
               'Resource' : ['BESS_1_SOC','BESS_1_P', 'BESS_1_Q','PV1_P', 'PV1_Q'] , 
               'Load' :['Load_1_P', 'Load_1_Q']
              },

    'BUS_2' : {'Voltage' : ['V_BUS2_A', 'V_BUS2_B', 'V_BUS2_C'] ,
               'Resource' : ['BESS_2_SOC', 'BESS_2_P','BESS_2_Q','PV2_P', 'PV2_Q'] , 
               'Load' :['Load_2_P', 'Load_2_Q']
              }
}

nominal_values= [1,1,1,50,0,0,0,0,0,0,1,1,1,50,0,0,0,0,0,0]







In [6]:
#########################################################################
#Start the environment(PSCAD)
actions= [0,0,0,0]
loads =[0,0,0,0,0,0]
BESS_1_SOC_prev_state = 50
runTimeFilePath = MG_env_path
print(runTimeFilePath)
start_PSCAD(runTimeFilePath, resultPath, projectName)
run_PSCAD(runTimeFilePath,resultPath, projectName, '', '', False, actions, loads,BESS_1_SOC_prev_state )

C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\
[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\AD-MYazdani\\Desktop\\SDSU Research-RL-Microgrid\\MG_env\\Output_Dest_Folder'
actions in the PSCAD_env  [0, 0, 0, 0]
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\


In [7]:
BESS_1_SOC_prev_state = 50
run_PSCAD(runTimeFilePath,resultPath, projectName, 'output_currentState.out', 'snap_currentState.snp', True, actions, loads,BESS_1_SOC_prev_state )

actions in the PSCAD_env  [0, 0, 0, 0]
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\


In [8]:
output_currentState_df = get_result(sourceFolder, 'output_currentState')
output_currentState_df.insert(loc=0, column='Time', value=output_currentState_df.index)
outputFile = os.path.join(resultPath,'state_1.csv')
output_currentState_df.to_csv(outputFile, index=False)
initial_state = avg_state_sample(output_currentState_df)
print(initial_state.shape)
output_currentState_df

(25,)


,Time,V_BUS1_A,V_BUS1_B,V_BUS1_C,V_BUS2_A,V_BUS2_B,V_BUS2_C,PV1_P,PV1_Q,BESS_2_SOC,...,BUS1_load_Q:1,BUS1_load_Q:2,BUS1_load_Q:3,BUS2_load_P,BUS2_load_Q,BESS_2_Q,BESS_2_P,BESS_1_Q,BESS_1_P,BESS_1_SOC
0,,,,,,,,,,,,,,,,,,,,,
0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,...,0.000000e+00,0.000000e+00,0.000000e+00,20.0,20.0,0.0,0.0,0.0,0.0,50.0
0.0003,0.0003,3.034999,-1.218962,-1.816037,2.188867,-0.905894,-1.298692,1.0,1.0,1.0,...,-2.086282e-23,-1.138811e-23,6.268231e-23,20.0,20.0,0.0,0.0,0.0,0.0,50.0
0.0006,0.0006,5.953411,-1.790185,-4.163227,4.549850,-1.473295,-3.138020,1.0,1.0,1.0,...,-4.563140e-22,-1.389267e-22,1.082714e-21,20.0,20.0,0.0,0.0,0.0,0.0,50.0
0.0009,0.0009,8.641133,-1.679357,-6.961775,6.834536,-1.536227,-5.422553,1.0,1.0,1.0,...,-2.925906e-21,-4.629268e-22,5.565152e-21,20.0,20.0,0.0,0.0,0.0,0.0,50.0
0.0012,0.0012,9.157486,-0.724421,-8.433064,7.690891,-0.929867,-6.911443,1.0,1.0,1.0,...,-1.002167e-20,-8.121569e-22,1.578363e-20,20.0,20.0,0.0,0.0,0.0,0.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.9986,1.9986,8.796220,-8.838869,0.042649,8.086429,-8.567157,0.426224,1.0,1.0,1.0,...,-6.098083e-32,6.971282e-32,5.435477e-32,20.0,20.0,0.0,0.0,0.0,0.0,50.0
1.9989,1.9989,9.318722,-8.212038,-1.106685,8.604620,-8.007523,-0.661629,1.0,1.0,1.0,...,-5.918980e-32,7.029883e-32,5.556090e-32,20.0,20.0,0.0,0.0,0.0,0.0,50.0
1.9992,1.9992,9.722156,-7.480278,-2.241878,9.012742,-7.345339,-1.741088,1.0,1.0,1.0,...,-5.752258e-32,7.047825e-32,5.703469e-32,20.0,20.0,0.0,0.0,0.0,0.0,50.0


In [9]:
# stateNumber ="1"
# outFileName = "state_"+stateNumber+".csv"
# outputFile  = os.path.join(resultPath, outFileName)
# print(outputFile)

        
# Result= get_result(sourceFolder, 'output_steadyState')      
# Result.insert(loc= 0, column='Time', value =Result.index)
# Result
# # Result.to_csv(r'C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\Output_Dest_Folder\state_1.csv', index = False)
# Result.to_csv(outputFile, index = False)


In [10]:
# pd.read_csv(resultPath+'\\'+'state_1.csv')

In [11]:
# import os
# currentdirectory = os.getcwd()
# print(currentdirectory)
# currentFile = os.path.join(currentdirectory, 'snap_currentState.snp')
# nextFile = os.path.join(currentdirectory, 'snap_nextState.snp')
# os.remove(currentFile)
# os.rename(nextFile, 'snap_currentState.snp')


In [12]:
# len(avg_state_sample(Result))

In [13]:
def dqn(n_episodes = 10 , max_t =15, eps_start= 1, eps_end = 0.01, eps_decay = 0.995):
    
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    
    scores = [] # The list of score from each timestep
    scores_window = deque(maxlen = 10)
    eps = eps_start
    
    # Each episode is equal to 1 day
    for i_episod in range(1,n_episodes+1):
        
        # Define a folder to get ride of the folders before start of each day
        
        #define a function in your environment to reset everything to their nominal values 
        state =  np.array(initial_state) # np.array(reset(model_topo,nominal_values))
                                            
        score = 0 
        
        for t in range(max_t): # max_t for me is 24 hours which is either 24*4 = 96(every 15 min) or 24*12=288 (every 5 min)
        
            timeStep = i_episod*t
            hours = int((timeStep%96)/4)
            hourQuarter = int((timeStep%96)%4)
            
            
            
            print("######","Day : ", i_episod, "----","time period : ", t, " Actual Time : ", hours, "--", hourQuarter, "######")
            action = agent.act(state, eps)                     # For now its only P,Q setpoints for BESS on bus1, # but the NN eventually should be designd to give P,Q for BESS on each bus
            action = action.tolist()[0]

        
            # Apply action in the PSCAD to get the next state(This happens in the MG_env.py -> step_env)
            next_state, reward , done = step_env(action, t+1, timeStep)   # PSCAD should  run in this line. 
                                                                # Technically, next_state is a df of samples (based on sampeling rate) which for now we take the average 
            
            
            # at this point you need to go to the source file and change snap_nextState --> snap_currentState 
            # so in when you start the next 't' itteration, PSCAD reads the most recent snap file for the input
            
            
            next_state = np.array(next_state)
            
            #print("next_state is")
            #print("The type is ",type(next_state))

            reward= mean(reward)
            
            agent.step(state,action, reward, next_state, done)
            #print("results of agent.step")
            
            state = next_state
            #print("state = next state")
            #print(state)
            score += int(reward)
            
            print("")
        
            if done:
                break
            
        
        scores.append(score)
        scores_window.append(score)
        eps = max(eps_end, eps_decay*eps)
        
        
        print('\rEpisode {} \tAverage Score: {:.2f}'.format(i_episod, score,np.mean(scores_window)), end="")
        if i_episod %3 ==0 :
            print('\rEpisode {} \tAverage score{:.2f}'.format(i_episod, np.mean(scores_window)))
        if np.mean(scores_window)>=10: # TBD based on env_reward
            print("save the torch model")
        
    
    return scores

In [ ]:
out_scores = dqn()

###### Day :  1 ---- time period :  0  Actual Time :  0 -- 0 ######
randomVal 0.28183784439970383  <=>  eps 1
Suggested action values from NN tensor([[0.5086, 0.1853, 0.1153, 0.4491]])
Load Reward- This function still requires work
Load Reward- This function still requires work
actions in the PSCAD_env  [1.747936725616455, -0.3936178982257843, -1.3816922903060913, -1.0249000787734985]
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\feeder4_BESS_CSI.gf46\snap_currentState.snp
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\feeder4_BESS_CSI.gf46\snap_nextState.snp
currentState Removed
RENAME COMPLETED

###### Day :  1 ---- time period :  1  Actual Time :  0 -- 1 ######
randomVal 0.47700977655271704  <=>  eps 1
Suggested action values from NN tensor([[0.5029, 0.1794, 0.1229, 0.4426]])
Load Reward- This function still requires work
Load Reward- This function still requires work
actions in the PSCA

C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_agent\DQN_agent.py:214: UserWarning: Using a target size (torch.Size([4, 1])) that is different to the input size (torch.Size([4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(Q_expected, Q_targets)


Load Reward- This function still requires work
Load Reward- This function still requires work
actions in the PSCAD_env  [0.39883360266685486, 0.12433968484401703, 0.9427483677864075, 1.0074639320373535]
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\feeder4_BESS_CSI.gf46\snap_currentState.snp
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\feeder4_BESS_CSI.gf46\snap_nextState.snp
currentState Removed
RENAME COMPLETED

###### Day :  1 ---- time period :  9  Actual Time :  2 -- 1 ######
randomVal 0.560600218853524  <=>  eps 1
Suggested action values from NN tensor([[0.5257, 0.2011, 0.1461, 0.4693]])
Load Reward- This function still requires work
Load Reward- This function still requires work
actions in the PSCAD_env  [-1.1262696981430054, 0.8614098429679871, -0.17577651143074036, -1.2189483642578125]
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\
C:\Users\AD-MYazdani\Desktop\SD


###### Day :  2 ---- time period :  4  Actual Time :  2 -- 0 ######
randomVal 0.10474628201641056  <=>  eps 0.995
Suggested action values from NN tensor([[0.5700, 0.2448, 0.1925, 0.5224]])
Load Reward- This function still requires work
Load Reward- This function still requires work
actions in the PSCAD_env  [0.7063958048820496, 1.4515633583068848, 1.3878865242004395, 1.317682147026062]
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\feeder4_BESS_CSI.gf46\snap_currentState.snp
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\feeder4_BESS_CSI.gf46\snap_nextState.snp
currentState Removed
RENAME COMPLETED
 Number of samples in memory: 10			 BATCH_SIZE: 4
Q_targets_next  tensor([[0.5043],
        [0.5023],
        [0.5031],
        [0.5045]])
Q_target is:  tensor([[10715.5273],
        [10714.0039],
        [10713.6875],
        [10714.2334]])
Q_expected is:  tensor([0.5691, 0.2452, 0.1919, 0.5224]

Load Reward- This function still requires work
Load Reward- This function still requires work
actions in the PSCAD_env  [-0.16314825415611267, -0.3699917495250702, 0.9609178900718689, -0.3686453402042389]
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\feeder4_BESS_CSI.gf46\snap_currentState.snp
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\feeder4_BESS_CSI.gf46\snap_nextState.snp
currentState Removed
RENAME COMPLETED

Episode 2 	Average Score: 160697.00save the torch model
###### Day :  3 ---- time period :  0  Actual Time :  0 -- 0 ######
randomVal 0.7548584132190378  <=>  eps 0.990025
Suggested action values from NN tensor([[0.6393, 0.3183, 0.2524, 0.6097]])
Load Reward- This function still requires work
Load Reward- This function still requires work
actions in the PSCAD_env  [0.6165337562561035, 1.20832097530365, -1.3448481559753418, -0.2032521665096283]
C:\Users\AD-MYazdani\Desktop\SDSU

 Number of samples in memory: 10			 BATCH_SIZE: 4
Q_targets_next  tensor([[0.5034],
        [0.5048],
        [0.5036],
        [0.5043]])
Q_target is:  tensor([[10714.0586],
        [10714.0879],
        [10714.0840],
        [10714.1816]])
Q_expected is:  tensor([0.6795, 0.3550, 0.3068, 0.6561], grad_fn=<SelectBackward>)
loss :  tensor(1.1478e+08, grad_fn=<MseLossBackward>)

###### Day :  3 ---- time period :  10  Actual Time :  7 -- 2 ######
randomVal 0.05699047999950346  <=>  eps 0.990025
Suggested action values from NN tensor([[0.7026, 0.3770, 0.3297, 0.6830]])
Load Reward- This function still requires work
Load Reward- This function still requires work
actions in the PSCAD_env  [1.079127311706543, -1.5131330490112305, 0.020647544413805008, -0.26166877150535583]
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\feeder4_BESS_CSI.gf46\snap_currentState.snp
C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Micro

In [ ]:
state = np.array(reset(model_topo,nominal_values))
print(state)
action = agent.act(state, 0) 
print(action)

In [ ]:
action = torch.tensor([[0,0,0,0],[0,0,0,0],[0,0,0,0],[1,0,1,1]])
a= torch.randn(4,4)
print(a)
test = torch.gather(a,1,action)
test[0]

In [ ]:
test2 = a.gather(1,action)
test2

In [ ]:
a.tolist()[0]

In [ ]:
# # add to the previouse block, Initializing the PSCAD, getting the snapshot SteadyState

# if os.path.exists(resultPath) and os.path.isdir(resultPath):
#     shutil.rmtree(resultPath)


# try:
#     os.mkdir(resultPath)
# except OSError as error:
#     print(error)
    

# pscad = mhi.pscad.launch()

# # pscad.load(r"C:\Users\AD-MYazdani\Desktop\SDSU Research-RL-Microgrid\MG_env\feeder4_BESS_CSI.pscx")
# testFilePath = os.path.join(currentDirectory+"\\MG_env",'feeder4_BESS_CSI.pscx')
# pscad.load(testFilePath)


# with mhi.pscad.application() as pscad:
#     mainProject = pscad.project(projectName)
#     mainProject.parameters( time_duration="2", time_step="100",sample_step="250",
#                                     StartType="0", PlotType="1", output_filename="output_steadyState.out",
#                                     SnapType="1", SnapTime="2", snapshot_filename="Snap_steadyState.snp"
                                   
#                            )
#     mainProject.run()



In [ ]:
timeStep = 192
